### Check RTX5090 running CUDA

In [1]:
import torch
import torchvision.models as models

# Check if CUDA
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
print(f"GPU count: {torch.cuda.device_count()}")

if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("❌ CUDA not detected by PyTorch")

# Test ResNet50 specifically
model = models.resnet50(pretrained=True).cuda()
test_batch = torch.randn(2, 3, 224, 224).cuda()
try:
    output = model(test_batch)
    print("ResNet50 works!")
except Exception as e:
    print(f"ResNet50 failed: {e}")

# Test EfficientNet
try:
    model_eff = models.efficientnet_b0(pretrained=True).cuda()
    output_eff = model_eff(test_batch)
    print("EfficientNet works!")
except Exception as e:
    print(f"EfficientNet failed: {e}")

PyTorch version: 2.10.0.dev20250922+cu128
CUDA available: True
CUDA version: 12.8
GPU count: 1
GPU name: NVIDIA GeForce RTX 5090
GPU memory: 31.8 GB


D:\Dropbox\AI Projects\buck\buck-env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\Dropbox\AI Projects\buck\buck-env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet50 works!
EfficientNet works!


D:\Dropbox\AI Projects\buck\buck-env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### Process deer data

In [2]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.models as models
import numpy as np
import cv2
import random
import json
import os
import glob
import itertools
import gc
from datetime import datetime
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# RTX 5090 Optimized Configuration
IMAGE_SIZE = (600, 600)  # Scaled up from 224x224
AUGMENTATION_TARGET = 1000
NUM_FOLDS = 10
NUM_WORKERS = 0
MIXED_PRECISION = True
COMPILE_MODEL = False

# Model configurations with appropriate batch sizes for 512x512 on RTX 5090
MODEL_CONFIGS = {
    # EfficientNet family
    'efficientnet_b0': {'model_fn': models.efficientnet_b0, 'batch_size': 128, 'freeze_layers': 3},
    'efficientnet_b1': {'model_fn': models.efficientnet_b1, 'batch_size': 96, 'freeze_layers': 3},
    'efficientnet_b2': {'model_fn': models.efficientnet_b2, 'batch_size': 80, 'freeze_layers': 3},
    'efficientnet_b3': {'model_fn': models.efficientnet_b3, 'batch_size': 64, 'freeze_layers': 3},
    'efficientnet_b4': {'model_fn': models.efficientnet_b4, 'batch_size': 48, 'freeze_layers': 3},
    'efficientnet_b5': {'model_fn': models.efficientnet_b5, 'batch_size': 32, 'freeze_layers': 3},
    'efficientnet_b6': {'model_fn': models.efficientnet_b6, 'batch_size': 24, 'freeze_layers': 3},
    'efficientnet_b7': {'model_fn': models.efficientnet_b7, 'batch_size': 16, 'freeze_layers': 3},
    
    # ResNet family
    'resnet18': {'model_fn': models.resnet18, 'batch_size': 256, 'freeze_layers': 2},
    'resnet34': {'model_fn': models.resnet34, 'batch_size': 192, 'freeze_layers': 2},
    'resnet50': {'model_fn': models.resnet50, 'batch_size': 128, 'freeze_layers': 2},
    'resnet101': {'model_fn': models.resnet101, 'batch_size': 80, 'freeze_layers': 2},
    'resnet152': {'model_fn': models.resnet152, 'batch_size': 64, 'freeze_layers': 2},
    
    # DenseNet family
    'densenet121': {'model_fn': models.densenet121, 'batch_size': 96, 'freeze_layers': 2},
    'densenet169': {'model_fn': models.densenet169, 'batch_size': 80, 'freeze_layers': 2},
    'densenet201': {'model_fn': models.densenet201, 'batch_size': 64, 'freeze_layers': 2},
    
    # MobileNet family
    'mobilenet_v2': {'model_fn': models.mobilenet_v2, 'batch_size': 192, 'freeze_layers': 3},
    'mobilenet_v3_small': {'model_fn': models.mobilenet_v3_small, 'batch_size': 256, 'freeze_layers': 3},
    'mobilenet_v3_large': {'model_fn': models.mobilenet_v3_large, 'batch_size': 192, 'freeze_layers': 3},
    
    # RegNet family
    'regnet_y_400mf': {'model_fn': models.regnet_y_400mf, 'batch_size': 128, 'freeze_layers': 2},
    'regnet_y_800mf': {'model_fn': models.regnet_y_800mf, 'batch_size': 96, 'freeze_layers': 2},
    'regnet_y_1_6gf': {'model_fn': models.regnet_y_1_6gf, 'batch_size': 80, 'freeze_layers': 2},
    'regnet_y_3_2gf': {'model_fn': models.regnet_y_3_2gf, 'batch_size': 64, 'freeze_layers': 2},
    
    # ConvNeXt family
    'convnext_tiny': {'model_fn': models.convnext_tiny, 'batch_size': 96, 'freeze_layers': 2},
    'convnext_small': {'model_fn': models.convnext_small, 'batch_size': 80, 'freeze_layers': 2},
    'convnext_base': {'model_fn': models.convnext_base, 'batch_size': 64, 'freeze_layers': 2},
    
    # MaxViT family
    'maxvit_t': {'model_fn': models.maxvit_t, 'batch_size': 48, 'freeze_layers': 2},
    
    # Swin Transformer family
    'swin_t': {'model_fn': models.swin_t, 'batch_size': 64, 'freeze_layers': 2},
    'swin_s': {'model_fn': models.swin_s, 'batch_size': 48, 'freeze_layers': 2},
    'swin_b': {'model_fn': models.swin_b, 'batch_size': 32, 'freeze_layers': 2},
    
    # Vision Transformer family
    'vit_b_16': {'model_fn': models.vit_b_16, 'batch_size': 64, 'freeze_layers': 6},
    'vit_b_32': {'model_fn': models.vit_b_32, 'batch_size': 96, 'freeze_layers': 6},
    'vit_l_16': {'model_fn': models.vit_l_16, 'batch_size': 32, 'freeze_layers': 8},
}

# Training hyperparameters
TRAINING_CONFIG = {
    'backbone_lr': 0.0001,
    'classifier_lr': 0.0005,
    'optimizer': 'adamw',
    'weight_decay': 0.05,
    'scheduler': 'cosine',
    'label_smoothing': 0.1,
    'dropout': 0.3,
    'max_epochs': 80,
    'patience': 25,
    'augmentation_strength': 'medium'
}

def detect_and_convert_image(image):
    if len(image.shape) == 2:
        return cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    elif len(image.shape) == 3:
        if image.shape[2] == 1:
            return cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        elif image.shape[2] == 3:
            return image
        elif image.shape[2] == 4:
            return cv2.cvtColor(image, cv2.COLOR_BGRA2RGB)
    return image

def load_combined_data():
    color_path = "D:\\Dropbox\\AI Projects\\buck\\images\\squared\\color\\*_NDA.png"
    gray_path = "D:\\Dropbox\\AI Projects\\buck\\images\\squared\\grayscale\\*_NDA.png"
    
    images = []
    ages = []
    sources = []
    
    print("Loading color images...")
    color_files = glob.glob(color_path)
    for img_path in color_files:
        try:
            img = cv2.imread(img_path)
            if img is None:
                continue
            
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = detect_and_convert_image(img)
            img_resized = cv2.resize(img, IMAGE_SIZE[::-1])
            
            filename = os.path.basename(img_path)
            filename_no_ext = os.path.splitext(filename)[0]
            parts = filename_no_ext.split('_')
            
            if len(parts) < 5:
                continue
            
            age_part = parts[3]
            if 'xpx' in age_part.lower() or 'p' not in age_part:
                continue
            
            try:
                age_value = float(age_part.replace('p', '.'))
                images.append(img_resized)
                ages.append(age_value)
                sources.append('color')
            except ValueError:
                continue
                
        except Exception as e:
            continue
    
    print(f"Loaded {len([s for s in sources if s == 'color'])} color images")
    
    print("Loading grayscale images...")
    gray_files = glob.glob(gray_path)
    for img_path in gray_files:
        try:
            img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
            if img is None:
                continue
            
            img = detect_and_convert_image(img)
            img_resized = cv2.resize(img, IMAGE_SIZE[::-1])
            
            filename = os.path.basename(img_path)
            filename_no_ext = os.path.splitext(filename)[0]
            parts = filename_no_ext.split('_')
            
            if len(parts) < 5:
                continue
            
            age_part = parts[3]
            if 'xpx' in age_part.lower() or 'p' not in age_part:
                continue
            
            try:
                age_value = float(age_part.replace('p', '.'))
                images.append(img_resized)
                ages.append(age_value)
                sources.append('grayscale')
            except ValueError:
                continue
                
        except Exception as e:
            continue
    
    print(f"Loaded {len([s for s in sources if s == 'grayscale'])} grayscale images")
    print(f"Total images: {len(images)}")
    
    ages_grouped = [5.5 if age >= 5.5 else age for age in ages]
    
    age_counts = Counter(ages_grouped)
    valid_ages = {age for age, count in age_counts.items() if count >= 3}
    
    filtered_images = []
    filtered_ages = []
    filtered_sources = []
    
    for img, age, source in zip(images, ages_grouped, sources):
        if age in valid_ages:
            filtered_images.append(img)
            filtered_ages.append(age)
            filtered_sources.append(source)
    
    print(f"Final dataset: {len(filtered_images)} images")
    print(f"Age distribution: {dict(Counter(filtered_ages))}")
    
    return np.array(filtered_images, dtype=np.uint8), filtered_ages, filtered_sources

def enhanced_augment_image(image, strength='medium'):
    if image.dtype != np.uint8:
        image = image.astype(np.uint8)
    
    if strength == 'light':
        rot_prob, flip_prob, bright_prob, gamma_prob, noise_prob = 0.5, 0.3, 0.6, 0.2, 0.1
        rot_range, bright_range = 8, (0.85, 1.15)
    elif strength == 'medium':
        rot_prob, flip_prob, bright_prob, gamma_prob, noise_prob = 0.7, 0.5, 0.8, 0.4, 0.3
        rot_range, bright_range = 12, (0.75, 1.25)
    else:  # heavy
        rot_prob, flip_prob, bright_prob, gamma_prob, noise_prob = 0.8, 0.6, 0.9, 0.5, 0.4
        rot_range, bright_range = 18, (0.65, 1.35)
    
    if random.random() < rot_prob:
        angle = random.uniform(-rot_range, rot_range)
        h, w = image.shape[:2]
        M = cv2.getRotationMatrix2D((w//2, h//2), angle, 1.0)
        image = cv2.warpAffine(image, M, (w, h))
    
    if random.random() < flip_prob:
        image = cv2.flip(image, 1)
    
    if len(image.shape) == 3 and image.shape[2] == 3 and random.random() < 0.3:
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        image = cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB)
    
    if random.random() < bright_prob:
        alpha = random.uniform(*bright_range)
        beta = random.randint(-20, 20)
        image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    
    if random.random() < gamma_prob:
        gamma = random.uniform(0.85, 1.15)
        inv_gamma = 1.0 / gamma
        table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
        image = cv2.LUT(image, table)
    
    if random.random() < noise_prob:
        noise = np.random.normal(0, 5, image.shape).astype(np.int16)
        image_int16 = image.astype(np.int16)
        noisy_image = np.clip(image_int16 + noise, 0, 255)
        image = noisy_image.astype(np.uint8)
    
    return image

class OptimizedDataset(Dataset):
    def __init__(self, base_images, labels, aug_strength='medium', target_per_class=1000, training=True):
        self.base_images = base_images
        self.labels = np.array(labels)
        self.aug_strength = aug_strength
        self.training = training
        self.target_per_class = target_per_class
        
        unique_classes = np.unique(labels)
        self.class_to_indices = {}
        for cls in unique_classes:
            self.class_to_indices[cls] = np.where(self.labels == cls)[0]
        
        self.num_classes = len(unique_classes)
        self.class_list = sorted(unique_classes)
        self.length = self.num_classes * self.target_per_class
        
        self.mean = np.array([0.485, 0.456, 0.406], dtype=np.float32).reshape(3, 1, 1)
        self.std = np.array([0.229, 0.224, 0.225], dtype=np.float32).reshape(3, 1, 1)
        
        print(f"Dataset: {self.length} samples from {len(base_images)} base images")
    
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        class_idx = idx // self.target_per_class
        within_class_idx = idx % self.target_per_class
        
        target_class = self.class_list[class_idx]
        available_indices = self.class_to_indices[target_class]
        
        base_idx = available_indices[within_class_idx % len(available_indices)]
        image = self.base_images[base_idx].copy()
        
        if self.training and within_class_idx >= len(available_indices):
            image = enhanced_augment_image(image, self.aug_strength)
        
        image = image.astype(np.float32) / 255.0
        if len(image.shape) == 3:
            image = image.transpose(2, 0, 1)
        
        if not self.training and random.random() < 0.5:
            image = np.flip(image, axis=2).copy()
        
        image = (image - self.mean) / self.std
        
        return torch.from_numpy(image.astype(np.float32)), target_class

class MultiModelTrainer:
    def __init__(self, num_classes, save_dir=None):
        self.num_classes = num_classes
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        if save_dir is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            self.save_dir = f"multimodel_512_{timestamp}"
        else:
            self.save_dir = save_dir
        
        os.makedirs(self.save_dir, exist_ok=True)
        self.results = []
        
        print(f"Using device: {self.device}")
        if torch.cuda.is_available():
            print(f"GPU: {torch.cuda.get_device_name()}")
            print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
            
            torch.backends.cudnn.benchmark = True
            torch.backends.cudnn.allow_tf32 = True
            torch.backends.cuda.matmul.allow_tf32 = True
            
            if MIXED_PRECISION:
                self.scaler = torch.amp.GradScaler('cuda')
            else:
                self.scaler = None
    
    def create_model(self, model_name, model_config):
        print(f"Loading {model_name}...")
        model = model_config['model_fn'](weights='DEFAULT')
        
        # Freeze layers based on model type
        freeze_layers = model_config.get('freeze_layers', 2)
        
        if hasattr(model, 'features'):  # EfficientNet, DenseNet, etc.
            layers_to_freeze = list(model.features.children())[:freeze_layers]
            for layer in layers_to_freeze:
                for param in layer.parameters():
                    param.requires_grad = False
            
            if hasattr(model, 'classifier'):
                original_features = model.classifier[-1].in_features if isinstance(model.classifier, nn.Sequential) else model.classifier.in_features
                model.classifier = nn.Sequential(
                    nn.Dropout(TRAINING_CONFIG['dropout']),
                    nn.Linear(original_features, 512),
                    nn.ReLU(inplace=True),
                    nn.Dropout(TRAINING_CONFIG['dropout'] * 0.5),
                    nn.Linear(512, 256),
                    nn.ReLU(inplace=True),
                    nn.Dropout(TRAINING_CONFIG['dropout'] * 0.25),
                    nn.Linear(256, self.num_classes)
                )
        
        elif hasattr(model, 'fc'):  # ResNet, RegNet
            if hasattr(model, 'layer1'):  # ResNet
                layers_to_freeze = [model.conv1, model.bn1]
                if freeze_layers >= 1:
                    layers_to_freeze.append(model.layer1)
                if freeze_layers >= 2:
                    layers_to_freeze.append(model.layer2)
            
            for layer in layers_to_freeze:
                for param in layer.parameters():
                    param.requires_grad = False
            
            original_features = model.fc.in_features
            model.fc = nn.Sequential(
                nn.Dropout(TRAINING_CONFIG['dropout']),
                nn.Linear(original_features, 512),
                nn.ReLU(inplace=True),
                nn.Dropout(TRAINING_CONFIG['dropout'] * 0.5),
                nn.Linear(512, 256),
                nn.ReLU(inplace=True),
                nn.Dropout(TRAINING_CONFIG['dropout'] * 0.25),
                nn.Linear(256, self.num_classes)
            )
        
        elif hasattr(model, 'head'):  # Vision Transformers, Swin, ConvNeXt
            # Freeze some transformer blocks
            if hasattr(model, 'encoder') and hasattr(model.encoder, 'layers'):  # ViT
                layers_to_freeze = list(model.encoder.layers.children())[:freeze_layers]
                for layer in layers_to_freeze:
                    for param in layer.parameters():
                        param.requires_grad = False
            elif hasattr(model, 'features'):  # ConvNeXt
                layers_to_freeze = list(model.features.children())[:freeze_layers]
                for layer in layers_to_freeze:
                    for param in layer.parameters():
                        param.requires_grad = False
            
            original_features = model.head.in_features
            model.head = nn.Sequential(
                nn.Dropout(TRAINING_CONFIG['dropout']),
                nn.Linear(original_features, 512),
                nn.ReLU(inplace=True),
                nn.Dropout(TRAINING_CONFIG['dropout'] * 0.5),
                nn.Linear(512, 256),
                nn.ReLU(inplace=True),
                nn.Dropout(TRAINING_CONFIG['dropout'] * 0.25),
                nn.Linear(256, self.num_classes)
            )
        
        frozen_params = sum(p.numel() for p in model.parameters() if not p.requires_grad)
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print(f"Parameters: {trainable_params:,} trainable, {frozen_params:,} frozen")
        
        return model.to(self.device)
    
    def get_optimizer(self, model):
        backbone_params = []
        classifier_params = []
        
        for name, param in model.named_parameters():
            if param.requires_grad:
                if any(keyword in name for keyword in ['classifier', 'fc', 'head']):
                    classifier_params.append(param)
                else:
                    backbone_params.append(param)
        
        param_groups = [
            {'params': backbone_params, 'lr': TRAINING_CONFIG['backbone_lr']},
            {'params': classifier_params, 'lr': TRAINING_CONFIG['classifier_lr']}
        ]
        
        return optim.AdamW(param_groups, weight_decay=TRAINING_CONFIG['weight_decay'], fused=True)
    
    def get_scheduler(self, optimizer):
        return optim.lr_scheduler.CosineAnnealingLR(
            optimizer, T_max=TRAINING_CONFIG['max_epochs'], eta_min=1e-6
        )
    
    def train_model(self, model, model_name, train_loader, val_loader, test_loader, fold_num):
        optimizer = self.get_optimizer(model)
        scheduler = self.get_scheduler(optimizer)
        criterion = nn.CrossEntropyLoss(label_smoothing=TRAINING_CONFIG['label_smoothing'])
        
        best_val_acc = 0.0
        best_train_acc = 0.0
        best_test_acc = 0.0
        patience_counter = 0
        best_state = None
        
        for epoch in range(TRAINING_CONFIG['max_epochs']):
            # Training
            model.train()
            train_correct = 0
            train_total = 0
            
            for images, labels in train_loader:
                images, labels = images.to(self.device), labels.to(self.device)
                
                optimizer.zero_grad()
                
                if MIXED_PRECISION and self.scaler:
                    with torch.amp.autocast('cuda'):
                        outputs = model(images)
                        loss = criterion(outputs, labels)
                    
                    self.scaler.scale(loss).backward()
                    self.scaler.step(optimizer)
                    self.scaler.update()
                else:
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                
                _, predicted = torch.max(outputs, 1)
                train_total += labels.size(0)
                train_correct += (predicted == labels).sum().item()
            
            # Validation
            model.eval()
            val_correct = 0
            val_total = 0
            
            with torch.no_grad():
                for images, labels in val_loader:
                    images, labels = images.to(self.device), labels.to(self.device)
                    
                    if MIXED_PRECISION:
                        with torch.amp.autocast('cuda'):
                            outputs = model(images)
                    else:
                        outputs = model(images)
                    
                    _, predicted = torch.max(outputs, 1)
                    val_total += labels.size(0)
                    val_correct += (predicted == labels).sum().item()
            
            train_acc = 100 * train_correct / train_total
            val_acc = 100 * val_correct / val_total
            
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                best_train_acc = train_acc
                patience_counter = 0
                best_state = model.state_dict().copy()
                
                # Test accuracy when validation improves
                model.eval()
                test_correct = 0
                test_total = 0
                
                with torch.no_grad():
                    for images, labels in test_loader:
                        images, labels = images.to(self.device), labels.to(self.device)
                        
                        if MIXED_PRECISION:
                            with torch.amp.autocast('cuda'):
                                outputs1 = model(images)
                                outputs2 = model(torch.flip(images, [3]))
                                outputs = (outputs1 + outputs2) / 2
                        else:
                            outputs1 = model(images)
                            outputs2 = model(torch.flip(images, [3]))
                            outputs = (outputs1 + outputs2) / 2
                        
                        _, predicted = torch.max(outputs, 1)
                        test_total += labels.size(0)
                        test_correct += (predicted == labels).sum().item()
                
                best_test_acc = 100 * test_correct / test_total
            else:
                patience_counter += 1
            
            scheduler.step()
            
            if epoch % 20 == 0:
                print(f"    Epoch {epoch}: Train {train_acc:.1f}%, Val {val_acc:.1f}%")
            
            if patience_counter >= TRAINING_CONFIG['patience']:
                print(f"    Early stopping at epoch {epoch}")
                break
        
        # Calculate multiplicative metric
        multiplicative_score = (best_val_acc / 100) * (best_test_acc / 100) * 10000
        
        # Save model
        save_path = os.path.join(self.save_dir, f"{model_name}_fold{fold_num}_val{best_val_acc:.1f}_test{best_test_acc:.1f}_mult{multiplicative_score:.1f}.pth")
        torch.save({
            'model_state_dict': best_state,
            'model_name': model_name,
            'fold_number': fold_num,
            'train_accuracy': best_train_acc,
            'validation_accuracy': best_val_acc,
            'test_accuracy': best_test_acc,
            'multiplicative_score': multiplicative_score,
            'image_size': IMAGE_SIZE,
            'training_config': TRAINING_CONFIG
        }, save_path)
        
        result = {
            'model_name': model_name,
            'fold': fold_num,
            'train_acc': best_train_acc,
            'val_acc': best_val_acc,
            'test_acc': best_test_acc,
            'multiplicative_score': multiplicative_score,
            'save_path': save_path
        }
        
        print(f"    {model_name} Fold {fold_num}: Train {best_train_acc:.1f}%, Val {best_val_acc:.1f}%, Test {best_test_acc:.1f}%, Mult {multiplicative_score:.1f}")
        
        return result
    
    def run_comprehensive_evaluation(self, images, ages, sources):
        print(f"COMPREHENSIVE MODEL EVALUATION - {len(MODEL_CONFIGS)} models, {NUM_FOLDS} folds")
        print(f"Image size: {IMAGE_SIZE[0]}x{IMAGE_SIZE[1]}")
        print(f"Total experiments: {len(MODEL_CONFIGS) * NUM_FOLDS}")
        
        unique_ages = sorted(list(set(ages)))
        label_mapping = {age: i for i, age in enumerate(unique_ages)}
        y_indices = np.array([label_mapping[age] for age in ages])
        
        print(f"Classes: {len(unique_ages)}")
        
        for model_name, model_config in MODEL_CONFIGS.items():
            print(f"\n{'='*60}")
            print(f"TESTING MODEL: {model_name.upper()}")
            print(f"Batch size: {model_config['batch_size']}")
            print(f"{'='*60}")
            
            for fold in range(1, NUM_FOLDS + 1):
                print(f"\n[Fold {fold}/{NUM_FOLDS}] {model_name}")
                
                try:
                    # Data splitting
                    X_train, X_test, y_train, y_test = train_test_split(
                        images, y_indices, test_size=0.2, random_state=fold * 42, stratify=y_indices
                    )
                    
                    X_train_final, X_val, y_train_final, y_val = train_test_split(
                        X_train, y_train, test_size=0.2, random_state=fold * 42 + 1, stratify=y_train
                    )
                    
                    # Create datasets
                    train_dataset = OptimizedDataset(X_train_final, y_train_final, 
                                                   TRAINING_CONFIG['augmentation_strength'], AUGMENTATION_TARGET, True)
                    val_dataset = OptimizedDataset(X_val, y_val, 'light', 200, False)
                    test_dataset = OptimizedDataset(X_test, y_test, 'light', 200, False)
                    
                    # Create data loaders
                    batch_size = model_config['batch_size']
                    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
                    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
                    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
                    
                    # Create and train model
                    model = self.create_model(model_name, model_config)
                    result = self.train_model(model, model_name, train_loader, val_loader, test_loader, fold)
                    self.results.append(result)
                    
                    # Cleanup
                    del model, train_dataset, val_dataset, test_dataset, train_loader, val_loader, test_loader
                    torch.cuda.empty_cache()
                    gc.collect()
                    
                except Exception as e:
                    print(f"    FAILED: {str(e)}")
                    torch.cuda.empty_cache()
                    gc.collect()
                    continue
        
        # Save comprehensive results
        self.save_results()
        self.print_summary()
    
    def save_results(self):
        results_path = os.path.join(self.save_dir, "comprehensive_results.json")
        with open(results_path, 'w') as f:
            json.dump(self.results, f, indent=2)
        print(f"\nResults saved to: {results_path}")
    
    def print_summary(self):
        print(f"\n{'='*80}")
        print("COMPREHENSIVE EVALUATION SUMMARY")
        print(f"{'='*80}")
        
        # Group by model
        model_results = {}
        for result in self.results:
            model_name = result['model_name']
            if model_name not in model_results:
                model_results[model_name] = []
            model_results[model_name].append(result)
        
        # Print summary for each model
        for model_name, results in model_results.items():
            if not results:
                continue
            
            avg_train = np.mean([r['train_acc'] for r in results])
            avg_val = np.mean([r['val_acc'] for r in results])
            avg_test = np.mean([r['test_acc'] for r in results])
            avg_mult = np.mean([r['multiplicative_score'] for r in results])
            
            print(f"{model_name:20} | Train: {avg_train:5.1f}% | Val: {avg_val:5.1f}% | Test: {avg_test:5.1f}% | Mult: {avg_mult:6.1f}")
        
        # Best overall model
        if self.results:
            best_result = max(self.results, key=lambda x: x['multiplicative_score'])
            print(f"\nBest model: {best_result['model_name']} (Fold {best_result['fold']})")
            print(f"Multiplicative score: {best_result['multiplicative_score']:.1f}")
            print(f"Val: {best_result['val_acc']:.1f}%, Test: {best_result['test_acc']:.1f}%")

def main():
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    torch.cuda.synchronize()
    
    print("RTX 5090 Comprehensive Model Evaluation")
    print("512x512 Images - Multiple Architectures")
    print("=" * 60)
    
    start_time = time.time()
    
    images, ages, sources = load_combined_data()
    
    trainer = MultiModelTrainer(num_classes=len(set(ages)))
    trainer.run_comprehensive_evaluation(images, ages, sources)
    
    elapsed = (time.time() - start_time) / 60
    print(f"\nComplete evaluation finished in: {elapsed:.1f} minutes")

if __name__ == "__main__":
    main()

RTX 5090 Comprehensive Model Evaluation
512x512 Images - Multiple Architectures
Loading color images...
Loaded 202 color images
Loading grayscale images...
Loaded 40 grayscale images
Total images: 242
Final dataset: 242 images
Age distribution: {2.5: 41, 3.5: 51, 4.5: 57, 5.5: 60, 1.5: 33}
Using device: cuda
GPU: NVIDIA GeForce RTX 5090
GPU Memory: 34.2 GB
COMPREHENSIVE MODEL EVALUATION - 17 models, 10 folds
Image size: 600x600
Total experiments: 170
Classes: 5

TESTING MODEL: MOBILENET_V2
Batch size: 192

[Fold 1/10] mobilenet_v2
Dataset: 5000 samples from 154 base images
Dataset: 1000 samples from 39 base images
Dataset: 1000 samples from 49 base images
Loading mobilenet_v2...
Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to C:\Users\aaron/.cache\torch\hub\checkpoints\mobilenet_v2-7ebf99e0.pth


100%|█████████████████████████████████████████████████████████████████████████████| 13.6M/13.6M [00:00<00:00, 40.4MB/s]


Parameters: 3,005,397 trainable, 6,960 frozen
    Epoch 0: Train 42.9%, Val 56.6%
    Epoch 20: Train 100.0%, Val 76.9%
    Early stopping at epoch 32
    mobilenet_v2 Fold 1: Train 99.7%, Val 80.5%, Test 61.8%, Mult 4974.9

[Fold 2/10] mobilenet_v2
Dataset: 5000 samples from 154 base images
Dataset: 1000 samples from 39 base images
Dataset: 1000 samples from 49 base images
Loading mobilenet_v2...
Parameters: 3,005,397 trainable, 6,960 frozen
    Epoch 0: Train 46.5%, Val 44.8%
    Epoch 20: Train 100.0%, Val 56.6%
    Epoch 40: Train 100.0%, Val 59.2%
    Early stopping at epoch 52
    mobilenet_v2 Fold 2: Train 100.0%, Val 62.3%, Test 63.4%, Mult 3949.8

[Fold 3/10] mobilenet_v2
Dataset: 5000 samples from 154 base images
Dataset: 1000 samples from 39 base images
Dataset: 1000 samples from 49 base images
Loading mobilenet_v2...
Parameters: 3,005,397 trainable, 6,960 frozen
    Epoch 0: Train 44.4%, Val 48.8%
    Epoch 20: Train 100.0%, Val 63.4%
    Epoch 40: Train 100.0%, Val 67.6%
 

100%|█████████████████████████████████████████████████████████████████████████████| 9.83M/9.83M [00:00<00:00, 32.0MB/s]


Parameters: 1,579,349 trainable, 5,072 frozen
    FAILED: mat1 and mat2 shapes cannot be multiplied (256x576 and 1024x512)

[Fold 2/10] mobilenet_v3_small
Dataset: 5000 samples from 154 base images
Dataset: 1000 samples from 39 base images
Dataset: 1000 samples from 49 base images
Loading mobilenet_v3_small...
Parameters: 1,579,349 trainable, 5,072 frozen
    FAILED: mat1 and mat2 shapes cannot be multiplied (256x576 and 1024x512)

[Fold 3/10] mobilenet_v3_small
Dataset: 5000 samples from 154 base images
Dataset: 1000 samples from 39 base images
Dataset: 1000 samples from 49 base images
Loading mobilenet_v3_small...
Parameters: 1,579,349 trainable, 5,072 frozen
    FAILED: mat1 and mat2 shapes cannot be multiplied (256x576 and 1024x512)

[Fold 4/10] mobilenet_v3_small
Dataset: 5000 samples from 154 base images
Dataset: 1000 samples from 39 base images
Dataset: 1000 samples from 49 base images
Loading mobilenet_v3_small...
Parameters: 1,579,349 trainable, 5,072 frozen
    FAILED: mat1 a

...

100%|███████████████████████████████████████████████████████████████████████████████| 108M/108M [00:01<00:00, 82.7MB/s]


Parameters: 28,058,764 trainable, 229,590 frozen
    Epoch 0: Train 53.0%, Val 59.1%
    Epoch 20: Train 99.9%, Val 72.5%
    Early stopping at epoch 33
    swin_t Fold 1: Train 99.9%, Val 76.1%, Test 58.0%, Mult 4413.8

[Fold 2/10] swin_t
Dataset: 5000 samples from 154 base images
Dataset: 1000 samples from 39 base images
Dataset: 1000 samples from 49 base images
Loading swin_t...
Parameters: 28,058,764 trainable, 229,590 frozen
    Epoch 0: Train 56.2%, Val 46.0%
    Epoch 20: Train 100.0%, Val 55.2%
    Early stopping at epoch 32
    swin_t Fold 2: Train 99.9%, Val 58.3%, Test 65.7%, Mult 3830.3

[Fold 3/10] swin_t
Dataset: 5000 samples from 154 base images
Dataset: 1000 samples from 39 base images
Dataset: 1000 samples from 49 base images
Loading swin_t...
Parameters: 28,058,764 trainable, 229,590 frozen
    Epoch 0: Train 57.2%, Val 54.3%
    Epoch 20: Train 100.0%, Val 65.6%
    Epoch 40: Train 100.0%, Val 69.1%
    Epoch 60: Train 100.0%, Val 64.5%
    Early stopping at epoch 70

100%|███████████████████████████████████████████████████████████████████████████████| 190M/190M [00:03<00:00, 63.1MB/s]


Parameters: 49,376,668 trainable, 229,590 frozen
    Epoch 0: Train 64.3%, Val 60.2%
    Epoch 20: Train 99.8%, Val 72.4%
    Early stopping at epoch 39
    swin_s Fold 1: Train 100.0%, Val 77.2%, Test 56.5%, Mult 4361.8

[Fold 2/10] swin_s
Dataset: 5000 samples from 154 base images
Dataset: 1000 samples from 39 base images
Dataset: 1000 samples from 49 base images
Loading swin_s...
Parameters: 49,376,668 trainable, 229,590 frozen
    Epoch 0: Train 62.8%, Val 46.8%
    Epoch 20: Train 99.9%, Val 52.5%
    Epoch 40: Train 100.0%, Val 53.0%
    Early stopping at epoch 52
    swin_s Fold 2: Train 100.0%, Val 56.5%, Test 65.0%, Mult 3672.5

[Fold 3/10] swin_s
Dataset: 5000 samples from 154 base images
Dataset: 1000 samples from 39 base images
Dataset: 1000 samples from 49 base images
Loading swin_s...
Parameters: 49,376,668 trainable, 229,590 frozen
    Epoch 0: Train 67.0%, Val 60.1%
    FAILED: CUDA error: unknown error
Search for `cudaErrorUnknown' in https://docs.nvidia.com/cuda/cuda-

AcceleratorError: CUDA error: unknown error
Search for `cudaErrorUnknown' in https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html for more information.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
